## Import libraries and dependencies


In [1]:

import optimiseprime.data_prep as dp
import optimiseprime.data_analysis as da


import datetime as dt

import pandas as pd
import yfinance as yf
import quantstats as qs
qs.extend_pandas()

## Request user for portfolio details, and fetch price data from yfinance Yahoo! Finance API

In [2]:
portfolio_choice = ""
while portfolio_choice not in [1, 2]:
    try:
        portfolio_choice = int(input(         
            "To analyse an existing portfolio, type 1\n"
            "To analyse a hypothetical portfolio - type 2\n"
        )
                              )
    except Exception:
        print('Error: Invalid Choice.\n')
        
# Get portfolio data from user depending on choice:

ticker_list = []

while len(ticker_list) == 0:
    if portfolio_choice == 1:
        existing_portfolio = dp.get_existing_portfolio()
        
        # Create a list of only the tickers
        ticker_list = []
        for key, value in existing_portfolio.items():
            ticker_list.append(key)
        ticker_list = pd.DataFrame(columns=ticker_list).add_suffix('-USD').columns.tolist()
        
    elif portfolio_choice == 2:
        ticker_list = dp.get_hypothetical_portfolio()
        # Request investment amount from user
        portfolio_value = dp.get_investment_amt()
        
    if len(ticker_list) == 0:
        print("You have not entered any tickers.")
    

To analyse an existing portfolio, type 1
To analyse a hypothetical portfolio - type 2
 2


Please enter the tickers of your cryptocurrencies one by one
Type 'done' when finished.


Ticker:  btc
Ticker:  eth
Ticker:  doge
Ticker:  done
How much do you wish to invest in total?
(Please input amount without currency symbol)
 20000


In [3]:
# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = dp.get_ticker_data(ticker_list)
portfolio_df.dropna(inplace = True)



In [4]:
# Keep only tickers in ticker_list for which data is available
ticker_list = [ticker for ticker in list(portfolio_df.columns.levels[0])]

# Print portfolio data for visual confirmation
if portfolio_choice == 1:    
    # Calculate portfolio value of each cryptocurrency held
    for ticker in existing_portfolio:
        existing_portfolio[ticker].append(
            {'value': portfolio_df[f"{ticker}-USD"].iloc[-1, 3] * existing_portfolio[ticker][0]['units']})
    print(f"--------------------------")                      
    print(f"Existing Portfolio:")
    portfolio_value = 0
    for ticker, units in existing_portfolio.items():
        value = existing_portfolio[ticker][1]['value']
        print(f"Value of {existing_portfolio[ticker][0]['units']} {ticker}: ${value:.2f}")
        portfolio_value += value
    print(f"\nTotal portfolio value: ${portfolio_value:.2f}\n")       
    
elif portfolio_choice == 2:
    print(f"--------------------------")                      
    print(f"Hypothetical Portfolio:")
    print(f"{[ticker.replace('-USD', '') for ticker in ticker_list]}")                   
    print(f"Investment amount:")
    print(f"${portfolio_value:.2f}\n")

print(
    f"NOTE:\n"
    f"To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.\n"
    f"While this ensures fair comparison of risk-reward metrics, it may compromise accuracy of these metrics if the sample sizes of historical price data are reduced.\n"
    f"Earliest date for which price data is available for all cryptocurrencies in your portfolio: {dt.datetime.date(portfolio_df.index[0])}"
)
print(f"--------------------------")

--------------------------
Hypothetical Portfolio:
['BTC', 'DOGE', 'ETH']
Investment amount:
$20000.00

NOTE:
To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.
While this ensures fair comparison of risk-reward metrics, it may compromise accuracy of these metrics if the sample sizes of historical price data are reduced.
Earliest date for which price data is available for all cryptocurrencies in your portfolio: 2019-01-13
--------------------------


## Data Analyses

### Calculate ratios:
* Sharpe ratio
* Sortino ratio
* Adjusted sortino ratio
* Gain to Pain ratio

In [5]:
# Calculate each of the following risk-reward ratio types
sharpe = da.calculate_sharpe_ratio(ticker_list, portfolio_df)
sortino =  da.calculate_sortino_ratio(ticker_list, portfolio_df)
adjusted_sortino = da.calculate_adjusted_sortino(ticker_list, portfolio_df)
gain_pain_ratio = da.calculate_gain_pain_ratio(ticker_list, portfolio_df)


# Store all ratios into a dict
ratios_df = pd.DataFrame(
    {
    'sharpe': sharpe,
    'sortino': sortino,
    'adj_sortino': adjusted_sortino,
    'gain_pain': gain_pain_ratio,
    }
)

# Calculate proportion scores for each risk-reward metric
weights = da.calculate_weights(ratios_df)



In [6]:
print(
    f"Portfolio allocation recommendations\n"
    f"Based on historical returns from {dt.datetime.date(portfolio_df.index[0])} to {dt.datetime.date(portfolio_df.index[-1])}"
)
print(f"Total portfolio value: ${portfolio_value:.2f}")
print(f"============================================================="
)

# Present all ratios in descending order

for column in ratios_df:
    if column == 'sharpe':
        da.sharpe_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'sortino':
        da.sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'adj_sortino':
        da.adj_sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'gain_pain':
        da.gain_pain_portfolio(ratios_df, weights, portfolio_value)
        


Portfolio allocation recommendations
Based on historical returns from 2019-01-13 to 2022-01-11
Total portfolio value: $20000.00

Metric: Sharpe Ratio
--------------
ETH-USD                            1.62 
Recommended % of total portfolio   37.67%
Recommended value allocation       $7533.75

BTC-USD                            1.48 
Recommended % of total portfolio   34.40%
Recommended value allocation       $6880.54

DOGE-USD                           1.20 
Recommended % of total portfolio   27.93%
Recommended value allocation       $5585.71


Metric: Sortino Ratio
--------------
DOGE-USD                           4.15 
Recommended % of total portfolio   47.37%
Recommended value allocation       $9473.86

ETH-USD                            2.40 
Recommended % of total portfolio   27.40%
Recommended value allocation       $5480.11

BTC-USD                            2.21 
Recommended % of total portfolio   25.23%
Recommended value allocation       $5046.02


Metric: Adjusted Sortino Rat

In [15]:
from matplotlib import pyplot
from statsmodels.tsa.ar_model import AutoReg
from math import sqrt
from statsmodels.graphics.tsaplots import plot_pacf
import panel as pn
import panel.widgets as pnw
import pandas as pd

import holoviews as hv
import hvplot.pandas



def forecast_portfolio(portfolio_returns, window = 60, foward_looking = 20, full_picture = False):
    # Visualising significant lags    
    returns_list = list(portfolio_returns.values)

    model = AutoReg(returns_list, lags = window, old_names=False)
    model_fit = model.fit()
    coef = model_fit.params

    index_list = [-num for num in sorted(list(range(1, window + 1)))]
    
    expected_list = []
    for look_foward in range(foward_looking):
        future_observation = coef[0]
        for back, coef_num in zip(index_list, coef[1:]):
            future_observation += (returns_list[back] * coef_num)
        expected_list.append(future_observation)
        returns_list.append(future_observation)
        
    if full_picture == True:
        forecasted_returns = pd.Series(returns_list).rename(portfolio_returns.name)
        return forecasted_returns
    else:
        expected_returns = pd.Series(expected_list).rename(portfolio_returns.name)
        return expected_returns

def forecast_all_portfolios(portfolio_returns_list, window, foward_looking, full_picture = False):
    series_list = []
    for portfolio_returns in portfolio_returns_list:
        series = forecast_portfolio(portfolio_returns, window, foward_looking, full_picture)
        series_list.append(series)
    portfolio_forecasted_returns = pd.concat(series_list, axis = 'columns', join = 'inner')
    return portfolio_forecasted_returns.hvplot()



In [19]:
ticker_close_names = []
for ticker in list(weights.index):
    ticker_close_names.append((ticker, 'close'))    
    
portfolio_close_df = portfolio_df[ticker_close_names]

portfolio_returns_list_df = []
for column_name in weights.columns:
    weights_portfolio = weights[column_name].values
    portfolio_returns = portfolio_close_df.dot(weights_portfolio)
    portfolio_returns = portfolio_returns.rename(column_name)
    portfolio_returns_list_df.append(portfolio_returns)


true_false = pn.widgets.Select(name='View whole graph', options=[True, False])

interact = pn.interact(forecast_all_portfolios, portfolio_returns_list = [portfolio_returns_list_df, True], window = range(100), foward_looking = range(100), full_picture = true_false)

pn.Column(interact[1], interact[0])

Column
    [0] Row
        [0] HoloViews(NdOverlay, name='interactive10449')
    [1] Column
        [0] Select(name='portfolio_returns_list', options=[[Date
2019-01-13     1266...], value=[Date
2019-01-13     1266....])
        [1] DiscreteSlider(formatter='%d', name='window', options=[0, 1, 2, 3, 4, ...], value=0)
        [2] DiscreteSlider(formatter='%d', name='foward_looking', options=[0, 1, 2, 3, 4, ...], value=0)
        [3] Select(name='View whole graph', options=[True, False], value=True)